In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import VOCDetection

import xmltodict
from PIL import Image
import numpy as np
from tqdm import tqdm

In [ ]:
class YOLO_PASCAL_VOC(VOCDetection):
    def __getitem__(self, index):
        """
        전처리
        결과 : X(image), y(label)
        """
        img = (Image.open(self.images[index]).convert('RGB').resize((224,224)))
        img_transform = transforms.Compose([
            transforms.PILToTensor(),
            transforms.Resize((224,224))
        ])
        img = torch.divide(img_transform(img), 255)
        
        target = xmltodict.parse(open(self.annotations[index].read()))
        
        classes = ["aeroplane", "bicycle", "bird", "boat", "bottle",
                   "bus", "car", "cat", "chair", "cow", "diningtable",
                   "dog", "horse", "motorbike", "person", "pottedplant",
                   "sheep", "sofa", "train", "tvmonitor"]
        
        label = np.zeros((7, 7, 25), dtype = float)
        # 격자 구조
        
        Image_Height = float(target['annotation']['size']['height'])
        Image_Width = float(target['annotation']['size']['width'])
        
        try:
            for i, obj in enumerate(target['annotation']['object']):
                if i >= 1:
                    break
    
                class_index = classes.index(obj['name'].lower())
                
                x_min = float(obj['bndbox']['xmin']) 
                y_min = float(obj['bndbox']['ymin'])
                x_max = float(obj['bndbox']['xmax']) 
                y_max = float(obj['bndbox']['ymax'])
                
                x_min = float((224.0/Image_Width)*x_min)
                y_min = float((224.0/Image_Height)*y_min)
                x_max = float((224.0/Image_Width)*x_max)
                y_max = float((224.0/Image_Height)*y_max)
                
                x = (x_min + x_max) / 2.0
                y = (y_min + y_max) / 2.0
                w = x_max - x_min
                h = y_max - y_min
                
                x_cell = int(x/32)
                y_cell = int(y/32)
                
                x_val_inCell = float((x - x_cell * 32.0)/32.0)
                y_val_inCell = float((y - y_cell * 32.0)/32.0)
                
                w = w / 224.0
                h = h / 224.0
                
                
                class_index_inCell = class_index + 5
                
                label[y_cell][x_cell][0] = x_val_inCell
                label[y_cell][x_cell][1] = y_val_inCell
                label[y_cell][x_cell][2] = w
                label[y_cell][x_cell][3] = h
                label[y_cell][x_cell][4] = 1.0
                label[y_cell][x_cell][class_index_inCell] = 1.0
                
    
        # Single-Object in Image
        except TypeError:
            obj = target['annotation']['object']
            class_index = classes.index(obj['name'].lower())
                
            x_min = float(obj['bndbox']['xmin']) 
            y_min = float(obj['bndbox']['ymin'])
            x_max = float(obj['bndbox']['xmax']) 
            y_max = float(obj['bndbox']['ymax'])

            x_min = float((224.0/Image_Width)*x_min)
            y_min = float((224.0/Image_Height)*y_min)
            x_max = float((224.0/Image_Width)*x_max)
            y_max = float((224.0/Image_Height)*y_max)

            x = (x_min + x_max)/2.0
            y = (y_min + y_max)/2.0
            w = x_max - x_min
            h = y_max - y_min

            x_cell = int(x/32) 
            y_cell = int(y/32) 
            x_val_inCell = float((x - x_cell * 32.0)/32.0) 
            y_val_inCell = float((y - y_cell * 32.0)/32.0)

            w = w / 224.0
            h = h / 224.0

            class_index_inCell = class_index + 5

            label[y_cell][x_cell][0] = x_val_inCell
            label[y_cell][x_cell][1] = y_val_inCell
            label[y_cell][x_cell][2] = w
            label[y_cell][x_cell][3] = h
            label[y_cell][x_cell][4] = 1.0
            label[y_cell][x_cell][class_index_inCell] = 1.0

        return img, torch.tensor(label)
            

In [ ]:
class YOLO(nn.Module):
    def __init__(self, VGG16):
        super().__init__()
        
        self.backbone = VGG16
        
        self.conv = nn.Sequential(
            nn.BatchNorm2d(512), # in_channels
            nn.Conv2d(in_channels = 512,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(1024),
            nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(1024),
            nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.BatchNorm2d(1024),
            nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.Flatten()
        )
        self.linear = nn.Sequential(
            nn.BatchNorm1d(7*7*1024),
            nn.Linear(7*7*1024, 4096),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 1470)
        )

    def forward(self, x):
        out = self.backbone(x)
        out = self.conv(out)
        out = self.linear(out)
        out = torch.reshape(out, (-1 ,7, 7, 30))
        return out

In [1]:
def yolo_multitask_loss(y_pred, y_true):
    """
    y_pred : [bounding_box_pred1, bounding_box_pred2, ...]
    y_true : [bounding_box, ...]
    """
    bath_loss = 0
    
    count = len(y_true)
    for i in range(0, count):
        """
        batch loop
        """
        y_true_unit = y_true[i].clone().detach().requires_gred_(True)
        y_pred_unit = y_pred[i].clone().detach().requires_grad_(True)
        y_true_unit = torch.reshape(y_true_unit, [49, 25])
        y_pred_unit = torch.reshape(y_pred_unit, [49, 30])

        loss = 0.0
        for j in range(0, len(y_true_unit)):
            """
            49: 격자
            """
            
            bbox1_pred = y_pred_unit[j, 0:4].clone().detach().requires_grad_(True)
            bbox1_pred_confidence = y_pred_unit[j, 4].clone().detach().requires_grad_(True)
            bbox2_pred = y_pred_unit[j, 5:9].clone().detach().requires_grad_(True)
            bbox2_pred_confidence = y_pred_unit[j, 9].clone().detach().requires_grad_(True)
            
            class_pred = y_pred_unit[j, 10:].clone().detach().requires_grad_(True)
            
            bbox_true = y_true_unit[j, :4].clone().detach().requires_grad_(True)
            bbox_true_confidence = y_true_unit[j, 4].clone().detach().requires_grad_(True)
            class_true = y_true_unit[j, 5:].clone().detach().requires_grad_(True)

            # IoU
            box_pred_1_np = bbox1_pred.detach().numpy()
            box_pred_2_np = bbox2_pred.detach().numpy()
            box_true_np = bbox_true.detach().numpy()

            bbox_true = y_true_unit[j, :4].clone().detach().requires_grad_(True)
            bbox_true_confidence = y_true_unit[j, 4].clone().detach().requires_grad_(True)
            class_true = y_true_unit[j, 5:].clone().detach().requires_grad_(True)

            # 바운딩 박스 좌표에서 [min_x, min_y, max_x, max_y]
            box_pred_1_minmax = np.asarray([box_pred_1_np[0] - 0.5*box_pred_1_np[2], box_pred_1_np[1] - 0.5*box_pred_1_np[3], box_pred_1_np[0] + 0.5*box_pred_1_np[2], box_pred_1_np[1] + 0.5*box_pred_1_np[3]])
            box_pred_2_minmax = np.asarray([box_pred_2_np[0] - 0.5*box_pred_2_np[2], box_pred_2_np[1] - 0.5*box_pred_2_np[3], box_pred_2_np[0] + 0.5*box_pred_2_np[2], box_pred_2_np[1] + 0.5*box_pred_2_np[3]])
            box_true_minmax   = np.asarray([box_true_np[0] - 0.5*box_true_np[2], box_true_np[1] - 0.5*box_true_np[3], box_true_np[0] + 0.5*box_true_np[2], box_true_np[1] + 0.5*box_true_np[3]])

            # [max(pred_x_min, true_x_min), max(pred_y_min, true_y_min), min(pred_x_max, true_x_max), min(pred_y_max, true_y_max)]
            InterSection_pred_1_with_true = [max(box_pred_1_minmax[0], box_true_minmax[0]), max(box_pred_1_minmax[1], box_true_minmax[1]), min(box_pred_1_minmax[2], box_true_minmax[2]), min(box_pred_1_minmax[3], box_true_minmax[3])]
            InterSection_pred_2_with_true = [max(box_pred_2_minmax[0], box_true_minmax[0]), max(box_pred_2_minmax[1], box_true_minmax[1]), min(box_pred_2_minmax[2], box_true_minmax[2]), min(box_pred_2_minmax[3], box_true_minmax[3])]


            IntersectionArea_pred_1_true = 0
            if (InterSection_pred_1_with_true[2] - InterSection_pred_1_with_true[0] + 1) >= 0 and (InterSection_pred_1_with_true[3] - InterSection_pred_1_with_true[1] + 1) >= 0 :
                  width = InterSection_pred_1_with_true[2] - InterSection_pred_1_with_true[0] + 1
                  height = InterSection_pred_1_with_true[3] - InterSection_pred_1_with_true[1] + 1
                  IntersectionArea_pred_1_true = width * height
            else:
                intersectionArea_pred_1_true = 0.0
              
            if (InterSection_pred_2_with_true[2] - InterSection_pred_2_with_true[0] + 1) >= 0 and (InterSection_pred_2_with_true[3] - InterSection_pred_2_with_true[1] + 1) >= 0 :
                IntersectionArea_pred_2_true = (InterSection_pred_2_with_true[2] - InterSection_pred_2_with_true[0] + 1) * (InterSection_pred_2_with_true[3] - InterSection_pred_2_with_true[1] + 1)
            
            Union_pred_1_true = box_pred_1_area + box_true_area - IntersectionArea_pred_1_true
            Union_pred_2_true = box_pred_2_area + box_true_area - IntersectionArea_pred_2_true

            IoU_box_1 = IntersectionArea_pred_1_true/Union_pred_1_true
            IoU_box_2 = IntersectionArea_pred_2_true/Union_pred_2_true

            responsible_box = torch.zeros(4)
            responsible_bbox_confidence = 0
            non_responsible_bbox_confidence = 0

            if IoU_box_1 >= IoU_box_2 :
                responsible_box = bbox1_pred.clone().detach().requires_grad_(True)
                responsible_bbox_confidence = bbox1_pred_confidence.clone().detach().requires_grad_(True)
                non_responsible_bbox_confidence = bbox2_pred_confidence.clone().detach().requires_grad_(True)
                                
            else :
                responsible_box = bbox2_pred.clone().detach().requires_grad_(True)
                responsible_bbox_confidence = bbox2_pred_confidence.clone().detach().requires_grad_(True)
                non_responsible_bbox_confidence = bbox1_pred_confidence.clone().detach().requires_grad_(True)
            
            obj_exist = torch.ones_like(bbox_true_confidence)
            if box_true_np[0] == 0.0 and box_true_np[1] == 0.0 and box_true_np[2] == 0.0 and box_true_np[3] == 0.0 : 
                obj_exist = torch.zeros_like(bbox_true_confidence) 

            localization_err_x = torch.pow( torch.subtract(bbox_true[0], responsible_box[0]), 2) # (x-x_hat)^2
            localization_err_y = torch.pow( torch.subtract(bbox_true[1], responsible_box[1]), 2) # (y-y_hat)^2

            localization_err_w = torch.pow( torch.subtract(torch.sqrt(bbox_true[2]), torch.sqrt(responsible_box[2])), 2) # (sqrt(w) - sqrt(w_hat))^2
            localization_err_h = torch.pow( torch.subtract(torch.sqrt(bbox_true[3]), torch.sqrt(responsible_box[3])), 2) # (sqrt(h) - sqrt(h_hat))^2

            weighted_localization_err = torch.multiply(localization_err, 5.0)    
            weighted_localization_err = torch.multiply(weighted_localization_err, obj_exist)

            class_confidence_score_obj = torch.pow(torch.subtract(responsible_bbox_confidence, bbox_true_confidence), 2)
            class_confidence_score_obj = torch.mul(class_confidence_score_obj, obj_exist)
            
            class_confidence_score_noobj = torch.pow(torch.subtract(non_responsible_bbox_confidence, torch.zeros_like(bbox_true_confidence)), 2)
            class_confidence_score_noobj = torch.multiply(class_confidence_score_noobj, 0.5)
            class_confidence_score_noobj = torch.mul(class_confidence_score_noobj, torchsubtract(torch.ones_like(obj_exist), obj_exist))

            class_confidence_score = torch.add(class_confidence_score_obj,  class_confidence_score_noobj) 


            classification_err = torch.pow(torch.subtract(class_true, class_pred), 2.0)
            classification_err = torch.sum(classification_err)
            classification_err = torch.multiply(classification_err, obj_exist)

            loss_OneCell_1 = torch.add(weighted_localization_err, class_confidence_score)
            
            loss_OneCell = torch.add(loss_OneCell_1, classification_err)

            if loss == 0 :
                loss = loss_OneCell.clone().detach().requires_grad_(True)
            else :
                loss = torch.add(loss, loss_OneCell)
        if batch_loss == 0:
            batch_loss = loss.clone().detach().require_grad_(True)
        else:
            batch_loss = torch.add(batch_loss, loss)
    batch_loss = torch.torch.divide(batch_loss, count)
    return batch_loss

In [ ]:
def train_YOLO(YOLO_model, criterion, optimizer, epochs, data_loader, device) :
    pbar = tqdm(range(epochs), desc="training", mininterval=0.01)

    for epoch in pbar:
        optimizer.param_groups[0]['lr']
        if epoch >=0 and epoch < 75 :
            optimizer.param_groups[0]['lr'] = 0.001 + 0.009 * (float(epoch)/(75.0))
        elif epoch >= 75 and epoch < 105 :
            optimizer.param_groups[0]['lr'] = 0.001
        else : 
            optimizer.param_groups[0]['lr'] = 0.0001
            
        for inputs, labels in data_loader:
			
            inputs = inputs.to(device)
            labels = labels.to(device)
			
            optimizer.zero_grad()
             
            outputs = YOLO_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward() 
            optimizer.step()

            # 배치로 학습시킬 때마다 loss 보여주기
            pbar_str = "training, [loss = %.4f]" % loss.item()
            pbar.set_description(pbar_str)
            
    return YOLO_model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' # gpu를 사용할지 cpu를 사용할지 결정합니다. 

# VOC 2007 dataset의 위치를 저장합니다. 
current_path = os.getcwd()
path2data = current_path + '/voc'
if not os.path.exists(path2data): # 만약 데이터셋이 저장되어 있지 않다면 
    os.mkdir(path2data) # 저장할 폴더를 생성합니다. 
    
BATCH_SIZE = 64
EPOCH = 135

# 데이터셋을 편하게 사용할 수 있는 데이터셋 클래스의 객체를 선언합니다.
# 만약 데이터셋이 없으면 해당 데이터셋이 있을 경로(path2data)에 데이터셋을 다운로드합니다.
Train_Dataset = YOLO_PASCAL_VOC(path2data, year='2007', image_set='train', download=True)
Test_Dataset = YOLO_PASCAL_VOC(path2data, year='2007', image_set='test', download=True)

# 생성한 데이터셋 객체로 DataLoader 객체를 생성합니다.
# DataLoader를 이용하면 미니배치 단위로 모델에 데이터를 넣는 것이 매우 쉬워집니다.
# 즉, 학습시키는 코드를 쉽게 구현할 수 있게 해줍니다.
data_loader = torch.utils.data.DataLoader(dataset=Train_Dataset, # 사용할 데이터셋
                                          batch_size=BATCH_SIZE, # 미니배치 크기
                                          shuffle=True, # 에포크마다 데이터셋 셔플할건가? 
                                          drop_last=True) # 마지막 배치가 BATCH_SIZE보다 작을 때, 마지막 배치를 사용하지 않으려면 True를, 사용할거면 False를 입력합니다. 
                                          
# 사전학습된 VGG16을 불러옵니다. 
VGGNet = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
# VGG16에 있는 레이어 중 제가 사용할 레이어들이 그레디언트 추적을 못하게 해 가중치 변화를 막고
# padding을 1로 만들어 same padding이 되게끔 만들어줍니다.
for i in range(len(VGGNet.features[:-1])) :
    if type(VGGNet.features[i]) == type(nn.Conv2d(64,64,3)) :
        VGGNet.features[i].weight.requires_grad = False
        VGGNet.features[i].bias.requires_grad = False
        VGGNet.features[i].padding = 1

# YOLO와 optimizer를 선언합니다. VGGNet.features[:-1]는 VGG16에서 제가 Backbone으로 사용할 레이어들을 말합니다.
YOLO_model =  YOLO(VGGNet.features[:-1]).to(device) # Create YOLO model
optimizer = torch.optim.SGD(YOLO_model.parameters(), lr = 0.01, momentum = 0.9, weight_decay=0.0005)

# 모델을 학습시킵니다.
YOLO_model = train_YOLO(YOLO_model, yolo_multitask_loss, optimizer, EPOCH, data_loader, device)

In [ ]:
lst = ['a', 'b', 'c']

try:
    lst.index('z')
except ValueError as e:
    pass

    model.eval()
    with torch.no_grad():
      

-1


In [ ]:
obj = {
    'name': '윤영로',
    'age': 24,
    'annotations': {
        'type': 'helo'
    }
}
obj['annotations']['type']

cell = [0, 1] x [0, 1]

'helo'

In [ ]:
'a' == 'A'.lower()

True